Info
https://notebooks.gesis.org/binder/jupyter/user/ipython-ipython-in-depth-p1yfe52k/notebooks/binder/Index.ipynb

Project
https://towardsdatascience.com/how-to-build-a-movie-recommendation-system-67e321339109


In [1]:
import pandas as pd
data = pd.read_csv("ml-25m/ratings.csv")
data

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [2]:
data = data.drop('timestamp', axis=1)
data.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


# Splitting the data

In [3]:
data.shape

(25000095, 3)

In [4]:
train_data = data.iloc[:int(data.shape[0]*0.80)]
test_data = data.iloc[int(data.shape[0]*0.80):]

In [5]:
train_data.shape

(20000076, 3)

In [6]:
test_data.shape

(5000019, 3)

# Matrix Factorization

In [46]:
from surprise import SVD
import numpy as np
import surprise
from surprise import Reader, Dataset
from scipy import sparse

In [8]:
# It is to specify how to read the dataframe.
# for our dataframe, we don't have to specify anything extra..
reader = Reader(rating_scale=(1,5))

# create the traindata from the dataframe...
train_data_mf = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

# build the trainset from traindata.., It is of dataset format from surprise library..
trainset = train_data_mf.build_full_trainset() 

In [9]:
# create the testata from the dataframe...
test_data_mf = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)

# build the testset from traindata.., It is of dataset format from surprise library..
testset = test_data_mf.build_full_trainset() 

In [10]:
svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [12]:
# Guradando las predicciones de entrenamiento y obteniendolas del trainset
train_preds = svd.test(trainset.build_testset())

train_pred_mf = np.array([pred.est for pred in train_preds])

In [14]:
train_pred_mf

array([4.49664335, 4.02944301, 3.84726905, ..., 3.80010959, 1.64490751,
       3.02036228])

In [15]:
# Guardando las predicciones de prueba y obteniendolas del testset
test_preds = svd.test(testset.build_testset())

test_pred_mf = np.array([pred.est for pred in test_preds])

In [16]:
test_pred_mf

array([3.59513273, 2.87269608, 2.81276874, ..., 2.37310274, 4.11912542,
       3.72471638])

# Preparando el data frame de entrenamiento

In [20]:
# Creando una matriz dispersa (sparse matrix)

train_sparse_matrix = sparse.csr_matrix((train_data.rating.values, (train_data.userId.values, train_data.movieId.values)))

In [24]:
#Promedio global de las peliculas por los usuarios
train_averages = dict()

#Obtener el promedio glob al the raitings in nuestro set de entrenamiento
train_global_average = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
train_averages['global'] = train_global_average
train_averages

{'global': 3.5336391971710506}

In [27]:
# Obtener el promedio de los usuarios in un diccionario (key: user_id / movie_id, value: abg raiting)

def get_average_raitings(sparse_matrix, of_users):
    
    # Promedio de ratings de usuarios/axes
    ax = 1 if of_users else 0 # 1 - User axes, 0 - Movie axes
    
    # ".A1" es para convertir Column_Matrix a 1-D numpy array
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Matriz booleana de matrices (Si un usuario calificó esa película o no)
    is_rated = sparse_matrix != 0
    # No de raitings de cada usuario o película
    no_of_ratings = is_rated.sum(axis=ax).A1
    
    # max_user y max_movie ids en una matriz dispersa
    u,m = sparse_matrix.shape
    # Crear un diccionario de usuarios y sus ratings promedio
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i] for i in range(u if of_users else m) if no_of_ratings[i] != 0}
    
    # Devovler el diccionario the ratings promedio
    return average_ratings

In [29]:
# Raiting promedio obtenido por un usario
train_averages['user'] = get_average_raitings(train_sparse_matrix, of_users=True)
print('\nRating promedio del usuario 25: ', train_averages['user'][25])


Rating promedio del usuario 25:  3.528735632183908


In [37]:
# Raiting promedio para una película
train_averages['movie'] = get_average_raitings(train_sparse_matrix, of_users=False)
print('\nRating promedio de la película 40: ', train_averages['movie'][40])


Rating promedio de la película 40:  3.6320907617504052


In [38]:
# Obtener usuarios, películas and ratings de nuestra matriz dispersa de meustras de entrenamiento
train_users, train_movies, train_ratings = sparse.find(train_sparse_matrix)

In [ ]:
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

final_data = pd.DataFrame()
count = 0

for (user, movie, rating) in zip(train_users, train_movies, train_ratings):
    st = datetime.now()
    
    # print(user, movie)
    
    # Ratings de "movie" por usuarios similares de "user"
    # Calcular los usuarios similares de "user"
    user_sim = cosine_similarity(train_sparse_matrix[user], train_sparse_matrix).ravel()
    top_sim_users = user_sim.argsort()[::-1][1:] # Estamos ignorando "El Usuario" de sus usuarios similares
    # Obtener los ratings de los usuarios mas similares para la película
    top_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
    # Haremos una longitud de "5" agregando promedio de películas
    top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_users_ratings.extend([train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
    
    # print(top_sim_users_ratings, end = " ")
    
    # Ratings por "user" para peliculas similares de "movie"
    # Calcular las películas similares de "movie"
    movie_sim = cosine_similarity(train_sparse_matrix[:, movie].T, train_sparse_matrix.T).ravel()
    top_sim_movies = movie_sim.argsort()[::-1][1:]
    # Obtenemos los ratings de la pelicula más similar calificada por el usuario
    top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
    top_sim_movies_ratings.extend([train_averages['user'][user]]*(5 - len(top_sim_movies_ratings)))
    
    # print(top_sim_movies_ratings, end = " : -- ")
    
    # Preparar la fila para ser guardado en un archivo
    row = list()
    row.append(user)
    row.append(movie)
    # Ahora le agregamos las otras caracteristicas a estos datos
    row.append(train_averages['global'])
    
    row.extend(top_sim_users_ratings)
    
    row.extend(top_sim_movies_ratings)
    
    row.append(train_averages['user'][user])
    
    row.append(train_averages['user'][movie])
    
    # Finalmente el rating actual del user-movie
    row.append(rating)
    count = count + 1
    final_data = final_data.append([row])
    
    if (count)%10000 == 0:
        print("Done for {} rows---- {}".format(count, datetime.now() - start))
    

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

...

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\U

C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])
C:\Users\IDEAPAD GAMING\AppData\Local\Temp\ipykernel_16992\167544638.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_data = final_data.append([row])


from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Calcula similitudes de usuarios y películas una vez
user_sim_matrix = cosine_similarity(train_sparse_matrix, train_sparse_matrix)
movie_sim_matrix = cosine_similarity(train_sparse_matrix.T, train_sparse_matrix.T)

final_data = []

start = datetime.now()

for (user, movie, rating) in zip(train_users, train_movies, train_ratings):
    st = datetime.now()
    
    # Ratings de "movie" por usuarios similares de "user"
    top_sim_users = np.argsort(user_sim_matrix[user])[::-1][1:] # Ignoramos el usuario actual
    top_sim_users_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
    top_sim_users_ratings = top_sim_users_ratings[top_sim_users_ratings != 0][:5]
    top_sim_users_ratings = np.concatenate([top_sim_users_ratings, [train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings))])
    
    # Ratings por "user" para películas similares de "movie"
    top_sim_movies = np.argsort(movie_sim_matrix[movie])[::-1][1:]
    top_sim_movies_ratings = train_sparse_matrix[user, top_sim_movies].toarray().ravel()
    top_sim_movies_ratings = top_sim_movies_ratings[top_sim_movies_ratings != 0][:5]
    top_sim_movies_ratings = np.concatenate([top_sim_movies_ratings, [train_averages['user'][user]]*(5 - len(top_sim_movies_ratings))])
    
    # Preparar la fila para ser guardada en un DataFrame
    row = [user, movie, train_averages['global']]
    row.extend(top_sim_users_ratings)
    row.extend(top_sim_movies_ratings)
    row.extend([train_averages['user'][user], train_averages['user'][movie], rating])
    final_data.append(row)
    
    count += 1
    if count % 10000 == 0:
        print("Done for {} rows---- {}".format(count, datetime.now() - start))